In [2]:
import pandas as pd
import requests
import orjson
import sqlite3
from bs4 import BeautifulSoup
from sqlalchemy import create_engine

# соединение к базе данных и создание таблицы
db = sqlite3.connect('hw1.db')
cursor = db.cursor()
cursor.execute('CREATE TABLE IF NOT EXISTS vacancies (company_name WARCHAR(200), position WARCHAR(200), job_description WARCHAR(2000), key_skills WARCHAR(200))')
db.commit()
sql_query = 'INSERT INTO vacancies (company_name, position, job_description, key_skills) VALUES (?, ?, ?, ?)'

#вытаскиваем данные из хх
headers = {
        'Accept-Encoding': 'gzip, deflate, br',
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36',
        'Accept': '*/*',
        'Connection': 'keep-alive',
    }
#выносим url как конст., загружаем одну старинцу со 100 вакансиями
#def get_vacancy_info(vac_result):
url= 'https://hh.ru/search/vacancy?no_magic=true&L_save_area=true&text=middle+Python+developer&search_field=name&excluded_text=&area=2&area=1&salary=&currency_code=RUR&experience=doesNotMatter&order_by=relevance&search_period=0&items_on_page=100'
result = requests.get(url, headers=headers) #загруженная старинца это result
if result.ok: # если 200, то парсим страницу
    soup = BeautifulSoup(result.content.decode(), 'lxml') #объект soup на основе загруженной Web-страницы с результатами поиска
    templates = orjson.loads(soup.find("template").contents[0].encode('utf-8')) #на консультации (чтобы не только 20 вакансий загрузилось)
    i=1
    for jobs in templates['vacancySearchResult']['vacancies']: # jobs в templates это все вакансии
        vac = jobs['links']['desktop'] #ссылки на вакансии в jobs это vac
        vac_result = requests.get(vac, headers=headers) #загружаем вакансию
        print(i, vac, vac_result.status_code) #выводим статус загруженной страницы вакансии
        if vac_result.ok: # если 200, то парсим страницу вакансии
            soup = BeautifulSoup(vac_result.content.decode(), 'lxml') #объект soup на основе загруженной Web-страницы с вакансией
            try:    #почему-то иногда вылезала ошибка в атрибуте text
                company_name = soup.find('span', attrs={'data-qa': 'bloko-header-2'}).text #company_name смотрим как текст в spane на ссылке компании справа
            except:
                print('Ошибка чтения названия компании')
            position = soup.find('h1').text #в soupe position смотрим в заголовке h1 как текст
            job_description = soup.find('div', attrs={'data-qa': 'vacancy-description'}).text #job_description берем в div vacancy-description
            #для key_skills
            skills_elements = soup.find_all('div', {'data-qa': 'bloko-tag bloko-tag_inline skills-element'})
            key_skills = ', '.join([i.text.strip() for i in skills_elements])
            try:
                cursor.execute(sql_query,[company_name , position , job_description , key_skills]) # сохраняем найденные данные в таблицу
                db.commit()
            except:
                print('Ошибка записи в базу данных!')
        i +=1
else:
    print(f'Ошибка загрузи страницы поиска. Статус загрузки: {result.status_code}') #если не 200

cursor.close()
db.close()


engine = create_engine('sqlite:///hw1.db')
df = pd.read_sql_query('SELECT * FROM vacancies', engine)
print("Количество вакансий в таблице:", len(df))
engine.dispose()

1 https://cheboksary.hh.ru/vacancy/83387552 200
2 https://cheboksary.hh.ru/vacancy/83386645 200
3 https://cheboksary.hh.ru/vacancy/83207801 200
4 https://cheboksary.hh.ru/vacancy/83460735 200
5 https://cheboksary.hh.ru/vacancy/83458412 200
6 https://cheboksary.hh.ru/vacancy/82040028 200
7 https://cheboksary.hh.ru/vacancy/82546684 200
8 https://cheboksary.hh.ru/vacancy/72687504 200
9 https://cheboksary.hh.ru/vacancy/83409678 200
10 https://cheboksary.hh.ru/vacancy/82778051 200
11 https://cheboksary.hh.ru/vacancy/81922954 200
12 https://cheboksary.hh.ru/vacancy/83300859 200
13 https://cheboksary.hh.ru/vacancy/82571318 200
14 https://cheboksary.hh.ru/vacancy/83279670 200
15 https://cheboksary.hh.ru/vacancy/82299017 200
16 https://cheboksary.hh.ru/vacancy/83315918 200
17 https://cheboksary.hh.ru/vacancy/83279671 200
18 https://cheboksary.hh.ru/vacancy/82573862 200
19 https://cheboksary.hh.ru/vacancy/83324797 200
20 https://cheboksary.hh.ru/vacancy/76868710 200
21 https://cheboksary.hh.ru/v